In [91]:
#run_name = "matteo_*_cumulative*"
#run_name = "matteo_?laps*"
#run_name = "rl_trained*"
#run_name = "matte_8laps_aio"
#run_name = "matteo_3laps"
#run_name = "matteo_2_cumulative0124"
result_path = "../Training/results"
training_step_length = 100_000

In [1]:
import glob
import json
import matplotlib.pyplot as plt
import os
from tensorflow.python.summary.summary_iterator import summary_iterator
import numpy as np

In [133]:
def generate_plotdict(evallist: list, split_length: int = 20) -> dict:
    plotdict = {}
    for eval in evallist:
        with open(eval) as e:
            tdict = {}
            for line in e.readlines():
                jdict = json.loads(line)
                if jdict["splitLength"] == split_length:
                    trackName = jdict["fileName"].split('-')[0]
                    if trackName not in tdict:
                        tdict[trackName] = []
                    tdict[trackName].append(jdict["result"])
        for key in tdict.keys():
            tdict[key] = [sum(tdict[key])/len(tdict[key])]
        
        for key, value in tdict.items():
            if key in plotdict.keys():
                plotdict[key] += value
            else:
                plotdict[key] = value
    return plotdict

def plot_dict(plotdict: dict, color_dict: dict, xrange = None, artist = None):
    if artist == None:
        artist = plt.subplot()
    if all(len(v) == 1 for v in plotdict.values()):
        artist.bar(list(plotdict.keys()), [v[0] for v in plotdict.values()], color=[color_dict[k] for k in plotdict.keys()])
    else:
        for track in plotdict.keys():
            if xrange is None:
                artist.plot(range(training_step_length, (len(plotdict[track]) + 1) * training_step_length, training_step_length), plotdict[track], label=track, color=color_dict[track])
            else:
                artist.plot(xrange, plotdict[track], label=track, color=color_dict[track])
        artist.legend()
    artist.set_ylim(0, 1)
    artist.set_title("Evaluation")

def extract_tfevents(path: str, tag: str = "Policy/Gail Reward"):
    plotlist = []
    tffile = glob.glob(f"{path}/Kart/events*")[0]
    for e in summary_iterator(tffile):
        for v in e.summary.value:
            if v.tag == tag:
                plotlist.append(v.simple_value)
    return plotlist

# SUPER EVALUATION-O-MATIC 9000

In [125]:
track_list = [f"Track{n}" for n in range(5)]
#track_list = ["Square", "8", "Monza", "Spa", "Twisty"]
color_list = [f"C{n}" for n in range(5)]
color_dict = {track_list[n]: color_list[n] for n in range(5)}

tag = "Policy/Gail Reward"

GAIL Reward / Evaluation, N per Lap

In [ ]:
run_name = "matteo_3laps"
split_length = 20

for trackN in range(5):
    evallist = []
    tflist = []
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    for run in glob.glob(f"{result_path}/{run_name}_Track{trackN}_*"):
        evallist.append(run + "/" + "evaluations.jsonl")
        tflist += extract_tfevents(run, tag)
   
    plot_dict(generate_plotdict(evallist, split_length), color_dict, artist = ax2)
    ax1.plot(np.linspace(1e5, 1e6, len(tflist)), tflist, label= track_list[trackN], color=color_list[trackN])
    ax1.legend()
    ax1.set_ylim(0, 1200)
    ax1.set_title(tag)
    fig.suptitle(run_name)
    plt.show()

Evaluation / Evaluation, N per Lap

In [ ]:
run_name = "matteo_3laps"

for trackN in range(5):
    evallist = []
    tflist = []
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    for run in glob.glob(f"{result_path}/{run_name}_Track{trackN}_*"):
        evallist.append(run + "/" + "evaluations.jsonl")
   
    plot_dict(generate_plotdict(evallist, 20), color_dict, artist = ax1)
    plot_dict(generate_plotdict(evallist, 40), color_dict, artist = ax2)
    fig.suptitle(run_name)
    plt.show()

GAIL Reward / Evaluation, Cumulative 0124

In [ ]:
run_name = "matteo_2_cumulative0124"

evallist = []
ax1 = plt.subplot(223)
ax2 = plt.subplot(224)

for trackN in range(4):
    trackName = track_list[[0, 1, 2, 4][trackN]]
    tflist = []

    axn = plt.subplot(2, 4, trackN + 1)

    for run in glob.glob(f"{result_path}/{run_name}_{trackN}"):
        evallist.append(run + "/" + "evaluations.jsonl")
        tflist += extract_tfevents(run, tag)    
    axn.plot(np.linspace(1e5, 1e6, len(tflist)), tflist, label=trackName, color=color_dict[trackName])
    axn.legend()
    axn.set_ylim(0, 1200)

plotdict = generate_plotdict(evallist)
ax1.bar(range(4), plotdict["Track3"], label="Track3", color=color_dict["Track3"])
ax2.bar(range(4), plotdict["Track3"], label="Track3", color=color_dict["Track3"])
ax1.set_ylim(0, 1)
ax2.set_ylim(0.55, 0.75)
ax1.legend()

ax1.indicate_inset_zoom(ax2)
plt.suptitle(run_name)
plt.show()

Evaluation / Evaluation, Cumulative 0124

In [ ]:
run_name = "matteo_2_cumulative0124"

evallist = []

for trackN in range(4):
    for run in glob.glob(f"{result_path}/{run_name}_{trackN}"):
        evallist.append(run + "/" + "evaluations.jsonl")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)

plotdict1 = generate_plotdict(evallist, 20)
plotdict2 = generate_plotdict(evallist, 40)

params1 = {"x": range(4), "height": plotdict1["Track3"], "label": "Track3", "color": color_dict["Track3"]}
params2 = {"x": range(4), "height": plotdict2["Track3"], "label": "Track3", "color": color_dict["Track3"]}

ax1.bar(**params1)
ax2.bar(**params2)
ax3.bar(**params1)
ax4.bar(**params2)

ax1.set_ylim(0, 1)
ax3.set_ylim(0.55, 0.75)
ax2.set_ylim(0, 1)
ax4.set_ylim(0.30, 0.50)

ax1.legend()
ax2.legend()

ax1.indicate_inset_zoom(ax3)
ax2.indicate_inset_zoom(ax4)

plt.suptitle(run_name)
plt.show()

## Evaluation Plot

The following cell loads all `evaluation<...>.json` files and plots their values.

In [ ]:
run_name = "matteo_?laps*"
xrange = np.linspace(3, 9, 60)

#run_name = "matteo_*_cumulative*"
#xrange = np.linspace(2, 11, 36)

split_length = 20

evallist = glob.glob(f"{result_path}/{run_name}/evaluations.jsonl")

plotdict = generate_plotdict(evallist, split_length)

plot_dict(plotdict, color_dict, xrange)

plt.show()

## Final Evaluation Plot

The following cell takes the last elements for each `run_name..._N` series of runs and plots their `evaluations.jsonl` files.

In [ ]:
run_name = "matteo_?laps*"
xrange = range(3,9)

#run_name = "matteo_*_cumulative*"
#xrange = range(2,11)

split_length = 20

all_evaluations = glob.glob(f"{result_path}/{run_name}/evaluations.jsonl")

all_runs = list(map(lambda evaljson: os.path.dirname(evaljson), all_evaluations))
highest_run_number = max(map(lambda run: run.split('_')[-1], all_runs))
all_last_runs = list(filter(lambda run: run.endswith(highest_run_number), all_runs))
evallist = [f"{run}/evaluations.jsonl" for run in all_last_runs]

plotdict = generate_plotdict(evallist, split_length)


plot_dict(plotdict, color_dict, xrange)

plt.show()